# Teil 1: Semantische Suche
PDF-Dokumente: Dir werden PDF-Dokumente mit definiertem Inhalt bereitgestellt.
Such-Tool Entwicklung: Erstelle ein Stück Software, in dem du Suchanfragen von Nutzern eingeben kannst. Es reicht dabei, wenn das im Code direkt passiert, es muss keine UI o.ä. entwickelt werden. Es ist ebenfalls kein Interface für das Hochladen oder Auswählen der PDFs nötig, auf diese kann einfach per Dateisystem an fixer Stelle zugegriffen werden.
Semantische Suche: Identifiziere relevante Abschnitte in den PDF-Dokumenten basierend auf den Benutzeranfragen mittels semantischer Suche. Es reicht dabei, wenn die produzierten Daten nur flüchtig vorgehalten werden, es muss nicht zwingend persistiert werden.
Antwort-Generierung: Generiere und präsentiere die Antworten mit Hilfe der Ergebnisse der semantischen Suche und des LLM. Es reicht auch hierbei ein einfacher Output auf stdout o.ä..


## Anmerkung:
Da wir hier viel mit tabellarischen Daten arbeiten, wäre hier ein multimodaler Ansatz nützlich. Also neben den extrahierten Texten auch die Tabellen und PDF seiten als Bilddaten zu speichern und mit der semantischen Suche zu verknüpfen. Alternativ kann man die Multimodalen Fähigkeiten von GPT-4 nutzen um aus den Bildern die Tabellen Daten zu extrahieren und in einer "object_map" an ein DataFrame zu verknüpfen.

### VectorStore und Index vorbereiten

In [1]:
from main import prepare_index, query
import os


persist_dir = "challenge_index"

index, storage = prepare_index([], load_dir=persist_dir)


#### Query ausführen

In [2]:
query_str = "Wie ist die Lebensdauer der XBO 2000 W/HTP XL OFR-Lampe?"
response = query(query_str, index)

# die Ergebnisse der semantischen Suche
# >>> response.source_nodes

# die Antworten
# >>> response.response

=== Antwort ===
3500 Stunden

=== Quellen (Top 2) ===
Metadata:
{'page_label': '2', 'file_name': 'ZMP_1007187_XBO_2000_W_HTP_XL_OFR.pdf', 'file_path': 'data\\coding_challenge\\ZMP_1007187_XBO_2000_W_HTP_XL_OFR.pdf', 'file_type': 'application/pdf', 'file_size': 136057, 'creation_date': '1980-01-01', 'last_modified_date': '2024-02-20', 'last_accessed_date': '2024-02-20', 'document_title': 'XBO 2000 W/HTP XL OFR-Lampenmodell Technische Daten', 'title': 'XBO 2000 W/HTP XL OFR-Lampenmodell Technische Daten', 'power': 2000, 'lifespan': 3500}
Quellcontent:
Technische Daten
Elektrische Daten
Nennstrom 70,00 A
Stromsteuerbereich 50…85 A
Nennleistung 2000,00 W
Nennspannung 28,0 V
Abmessungen & Gewicht
 
Durchmesser 52,0 mm
Länge 375,0 mm
Länge mit Sockel jedoch ohne Sockelstift 322,00 mm
Abstand Lichtschwerpunkt (LCL) 142,5 mm 1)
Kabel-/Leitungslänge, Eingangsseite -
Elektrodenabstand kalt 5,8 mm
Produktgewicht 452,00 g
1) Abstand Sockelboden zu Elektrodenspitze (kalt)
Temperaturen & Betriebsbed

# Teil 2: Skalierung
Funktioniert deine Lösung auch für 500 PDFs? Was gibt es bei so vielen PDFs zu beachten? Wo sind bottlenecks oder etwaige Probleme zu erwarten?
Wie würde man es lösen, wenn der Nutzer fragt: ‘Gebe mir alle Leuchtmittel mit mindestens 1500W und einer Lebensdauer von mehr als 3000 Stunden’?

## Antwort Skalierbarkeit:

In der Lösung zu Teil 1 wurde eine lokaler VectorStore benutzt der die Daten im Arbeitsspeicher hält. Dies ist nicht skalierbar, da der Arbeitsspeicher begrenzt ist.

Um die Skalierbarkeit zu gewährleisten würde man hier auf einen Anbieter wie Pinecone zurückgreifen. Dafür habe ich der Initialisierungs Funktion ein Argument hinzugefügt, dass es ermöglicht einen anderen VectorStore zu initialisieren. 

```python
from scripts.core import initialize_storage, initialize_index
from llama_index.vector_stores.pinecone import PineconeVectorStore
import pinecone

# ... load the documents ...

# init pinecone
pinecone.init(api_key="<api_key>", environment="<environment>")
pinecone.create_index(
    "collection", dimension=1536, metric="euclidean", pod_type="p1"
)

vector_store = PineconeVectorStore(pinecone.Index("quickstart"))

storage = initialize_storage(vector_store=vector_store)
index = initialize_index(storage, docs)

```

### Bottlenecks:

Beim indexieren der Dokumente und beim erstellen der Vektoren:
- Memory: Die Datenmenge kann nicht im Arbeitsspeicher gehalten werden
- CPU: Die Berechnung der Vektoren kann sehr rechenintensiv sein
- Rate Limit bei embedding Erstellung seitens OpenAI

Beim Suchen:
- Irrlevante Ergebnisse: Die semantische Suche kann auch irrelevante Ergebnisse liefern
- Performance: Die Suche kann sehr langsam sein sofern die Datenmenge auf lokale Hardware beschränkt ist



# Teil 2.1: Metadata Filter

Um die Anfrage "Gebe mir alle Leuchtmittel mit mindestens 1500W und einer Lebensdauer von mehr als 3000 Stunden" zu bearbeiten, wäre gerade im Kontext hoher Anzahl an Dokumenten ein Filter auf Metadaten sinnvoll. Alternativ könnte man die selbe Frage auch im Postprocessing Schritt filtern. Jedoch wäre das nicht gerade kosteneffizient.
Pinecone bietet die Möglichkeit direkt über operatoren "$gt", "$lt", "$eq", etc. zu filtern.
In der Lösung zu Teil 1 mit lokalem llama_index VectorStoreIndex ist zurzeit nur exakte Suche möglich.

Ich habe dafür einen Workaround implementiert unter scripts/metadata_filtering.py
Neben der reinen filterung muss auch die Anfrage per LLM auf mögliche Metadaten Filter geprüft werden.
Dazu ist ebenfalls eine Lösung implementiert unter scripts/metadata_filtering.py mit der Nutzung von Langchain.

## Next Steps:
Um die Anfrage korrekt zu Beantworten, müsste man mit höherem top_k Wert die Ergebnisse der semantischen Suche nochmals im Postprocessing Schritt besser aufbereiten. Dies geht auch gut mit llama_index.




In [3]:
from main import query_with_metadata

query_str = "Gebe mir alle Leuchtmittel mit mindestens 1500W und einer Lebensdauer von mehr als 3000 Stunden"

response = query_with_metadata(query_str, index)

=== Antwort ===
XBO 1600W XL OFR, XBO 2000 W/HTP XL OFR-Lampenmodell

=== Quellen (Top 2) ===
Metadata:
{'page_label': '2', 'file_name': 'ZMP_1007179_XBO_1600_W_XL_OFR.pdf', 'file_path': 'data\\coding_challenge\\ZMP_1007179_XBO_1600_W_XL_OFR.pdf', 'file_type': 'application/pdf', 'file_size': 134098, 'creation_date': '1980-01-01', 'last_modified_date': '2024-02-20', 'last_accessed_date': '2024-02-20', 'document_title': 'OSRAM XBO 1600W XL OFR Technical Specifications and Product Information', 'title': 'OSRAM XBO 1600W XL OFR Technical Specifications and Product Information', 'power': 1600, 'lifespan': 3500}
Quellcontent:
Technische Daten
Elektrische Daten
Nennstrom 65 A
Stromsteuerbereich 45…75 A
Nennleistung 1600,00 W
Nennspannung 24,0 V
Abmessungen & Gewicht
 
Durchmesser 52,0 mm
Länge 370,0 mm
Länge mit Sockel jedoch ohne Sockelstift 322,00 mm
Abstand Lichtschwerpunkt (LCL) 143,0 mm 1)
Kabel-/Leitungslänge, Eingangsseite -
Elektrodenabstand kalt 4,8 mm
Produktgewicht 418,00 g
1) Abst

# Teil 3: Verifizierung
Welche Möglichkeiten gibt das Ergebnis zu evaluieren / bewerten, ohne dass das zwangsweise von einem Menschen passiert? Welche Metriken / Möglichkeiten existieren, um die Ausgabe mit der Eingabe zu vergleichen und sicherzustellen, dass die Antwort zur Eingabe passt (Gerade bei konkreten Werten, wie Lebensdauer von mehr als 3000 Stunden...)

## Antwort:

Metrics:
Typischerweise werden für die Evaluierung von semantischen Suchen Metriken wie Correctness, Faithfullness, Relevancy. Dies geschieht mit einer Reihe von darauf optimierten Prompts. Also die Ergebnisse der RAG-Pipeline werden wieder mit einem LLM in der Rolle der Judge oder Jury evaluiert.
Dabei kann eine End-to-End Evaluierung durchgeführt werden, bei der die Anfrage, die Antwort und die Dokumente mit denen die Antwort generiert wurde, betrachtet werden.
Alternativ können auch die einzelnen Komponenten der RAG-Pipeline evaluiert werden.
So ist bei der semantischen Suche die Relevanz der gefundenen Dokumente ein wichtiger Faktor.
Bei der Antwortgenerierung ist die Qualität der Antwort ein wichtiger Faktor der sich mit Correctness und Faithfullness (nicht halluzinieren).
Das Ergebniss multipler Evaluationsanfragen kann dann mit Precision, Recall und F1-Score evaluiert werden.

Automatisierte Evaluierung:
- Nutzung von Testdatensets wo Ground Truth bekannt ist
- Automatisierte Generierung von Anfragen und Generierung von Ground Truth über Majortiy Voting (GPT-4)

